In [ ]:
import pandas as pd
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pygmt as gmt
import xarray as xr

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

from scripts import morphoGrid as morph
from scipy.spatial import cKDTree

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Get exposed shelf percentage

In [ ]:
file = "shelfExpose.csv"
data = pd.read_csv(
            file,
            sep=r",",
            engine="c",
            na_filter=False,
            dtype=np.float,
            low_memory=False,
        )

time = data.values[:, 0]
low = data.values[:, 1]*100.
mid = data.values[:, 2]*100.
high = data.values[:, 3]*100.

### Intermittent flooding

+ -415 (584) -400 (600) -393 (607)
+ -337 (663) -325 (675) -304 (696) 
+ -246 (754) -237 (763) -230 (770)
+ -220 (780) -213 (787) -190 (810)
+ -107 (893) -81 (919)  -70 (930)

### Full flooding

+ -132 (868) -124 (876) -112 (888)
+ -16 (984) 0 (1000)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10,5))

plt.plot(time,low,color='k', ls='-', lw=0.5)
plt.plot(time,mid,color='k', ls='-.', lw=1.5)
plt.plot(time,high,color='k', ls='-', lw=0.5)


ax.fill_between(time, low, high, color='grey', alpha=0.2)

yy = "#ffeaa8"
bb = "royalblue"
gg = "#008a73"

vv = np.zeros(len(time))
# vv = np.ones(len(time[0:874]))*100.
# ax.fill_between(time[0:874], vv, high[0:874], color=yy, alpha=0.4)

ax.fill_between(time[0:584], vv[0:584], low[0:584], color=yy, alpha=0.8)
ax.fill_between(time[584:607], vv[584:607], low[584:607], color=gg, alpha=0.8)
ax.fill_between(time[607:663], vv[607:663], low[607:663], color=yy, alpha=0.8)
ax.fill_between(time[663:696], vv[663:696], low[663:696], color=gg, alpha=0.8)
ax.fill_between(time[696:754], vv[696:754], low[696:754], color=yy, alpha=0.8)
ax.fill_between(time[754:770], vv[754:770], low[754:770], color=gg, alpha=0.8)
ax.fill_between(time[770:780], vv[770:780], low[770:780], color=yy, alpha=0.8)
ax.fill_between(time[780:810], vv[780:810], low[780:810], color=gg, alpha=0.8)
ax.fill_between(time[810:868], vv[810:868], low[810:868], color=yy, alpha=0.8)
ax.fill_between(time[868:888], vv[868:888], low[868:888], color=bb, alpha=0.8)
ax.fill_between(time[888:893], vv[888:893], low[888:893], color=yy, alpha=0.8)
ax.fill_between(time[893:930], vv[893:930], low[893:930], color=gg, alpha=0.8)
ax.fill_between(time[930:984], vv[930:984], low[930:984], color=yy, alpha=0.8)
ax.fill_between(time[984:-1], vv[984:-1], low[984:-1], color=bb, alpha=0.8)

# vv = np.ones(len(time[874:884]))*100.
# ax.fill_between(time[874:884], vv, high[874:884], color=bb, alpha=0.7)

# vv = np.zeros(len(time[874:884]))
# ax.fill_between(time[874:884], vv, low[874:884], color=bb, alpha=0.7)

# vv = np.zeros(len(time[884:990]))
# ax.fill_between(time[884:990], vv, low[884:990], color=yy, alpha=0.4)

# vv = np.ones(len(time[884:990]))*100
# ax.fill_between(time[884:990], vv, high[884:990], color=yy, alpha=0.4)

# vv = np.ones(len(time[990:-1]))*100
# ax.fill_between(time[990:-1], vv, high[990:-1], color=bb, alpha=0.7)

# vv = np.zeros(len(time[990:-1]))
# ax.fill_between(time[990:-1], vv, low[990:-1], color=bb, alpha=0.7)



# # plt.grid()
plt.xlim(-1000,0)
plt.ylim(0,110)
plt.xlabel('Time (kyr BP)')
plt.ylabel('Percentage of exposed sundaland shelf')
fig.tight_layout()
fig.savefig('perc.pdf')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10,5))

plt.plot(time,low,color='k', ls='-', lw=0.5)
plt.plot(time,mid,color='k', ls='-.', lw=1.5)
plt.plot(time,high,color='k', ls='-', lw=0.5)


ax.fill_between(time, low, high, color='grey', alpha=0.4)

# # plt.grid()
plt.xlim(-450,0)
plt.ylim(0,110)
plt.xlabel('Time (kyr BP)')
plt.ylabel('Percentage of exposed sundaland shelf')
fig.tight_layout()
fig.savefig('zoomperc.pdf')

# Get biodiversity data

In [ ]:
file = "data/bio.csv"
biodata = pd.read_csv(
            file,
            sep=r",",
            engine="c",
            na_filter=False,
            low_memory=False,
        )

In [ ]:
biotime = -biodata['tmean'].values*1000

In [ ]:
417.525-45.43

In [ ]:
biodata

In [ ]:

import matplotlib.patches as patches


rect1 = patches.Rectangle((-157,0),98,2.6,facecolor='tab:orange', alpha=0.8)
rect1s = patches.Rectangle((-157,0),98,2.6,facecolor='None', linewidth=1, edgecolor='k')
rect2 = patches.Rectangle((-886,3.2),493,2.6,facecolor='tab:orange', alpha=0.8)
rect2s = patches.Rectangle((-886,3.2),493,2.6,edgecolor='k', linewidth=1,facecolor='None')
rect3 = patches.Rectangle((-845,6.3),815,2.6,facecolor='tab:orange', alpha=0.8)
rect3s = patches.Rectangle((-845,6.3),815,2.6,edgecolor='k', linewidth=1,facecolor='None')
rect4 = patches.Rectangle((-1300,9.4),732,2.6,facecolor='tab:orange', alpha=0.8)
rect4s = patches.Rectangle((-1300,9.4),732,2.6,edgecolor='k', linewidth=1,facecolor='None')
rect5 = patches.Rectangle((-332,12.5),197,2.6,facecolor='tab:orange', alpha=0.8)
rect5s = patches.Rectangle((-332,12.5),197,2.6,edgecolor='k', linewidth=1,facecolor='None')


rect6 = patches.Rectangle((-470,16.4),150,2.6,facecolor='tab:cyan', alpha=0.9)
rect6s = patches.Rectangle((-470,16.4),150,2.6,edgecolor='k', linewidth=1,facecolor='None')




rect7 = patches.Rectangle((-435,20.3),144,2.6,facecolor='tab:gray', alpha=0.7)
rect7s = patches.Rectangle((-435,20.3),144,2.6,edgecolor='k', linewidth=1,facecolor='None')








rect8 = patches.Rectangle((-417.525,24.1),372,2.6,facecolor='tab:green', alpha=0.7)
rect8s = patches.Rectangle((-417.525,24.1),372,2.6,edgecolor='k', linewidth=1,facecolor='None')



fig,ax = plt.subplots(1, 1, figsize=(10,4))

# Add the patch to the Axes
ax.add_patch(rect1)
ax.add_patch(rect1s)
ax.add_patch(rect2)
ax.add_patch(rect2s)
ax.add_patch(rect3)
ax.add_patch(rect3s)
ax.add_patch(rect4)
ax.add_patch(rect4s)
ax.add_patch(rect5)
ax.add_patch(rect5s)
ax.add_patch(rect6)
ax.add_patch(rect6s)
ax.add_patch(rect7)
ax.add_patch(rect7s)
ax.add_patch(rect8)
ax.add_patch(rect8s)


xx = [biotime[0],biotime[0]]
yy = [0.1,2.5]
plt.plot(xx,yy,lw=2,c='k')

xx = [biotime[1],biotime[1]]
yy = [3.2,5.7]
plt.plot(xx,yy,lw=2,c='k')

xx = [biotime[2],biotime[2]]
yy = [6.4,8.8]
plt.plot(xx,yy,lw=2,c='k')

xx = [biotime[3],biotime[3]]
yy = [9.5,11.9]
plt.plot(xx,yy,lw=2,c='k')

xx = [biotime[4],biotime[4]]
yy = [12.6,15]
plt.plot(xx,yy,lw=2,c='k')

# xx = [-1000,0]
# yy = [15.7,15.7]
# plt.plot(xx,yy,lw=1,ls='-',c='k')

xx = [biotime[5],biotime[5]]
yy = [16.44,18.8]
plt.plot(xx,yy,lw=2,c='k')

# xx = [-1000,0]
# yy = [19.6,19.6]
# plt.plot(xx,yy,lw=1,ls='-',c='k')

xx = [biotime[6],biotime[6]]
yy = [20.34,22.74]
plt.plot(xx,yy,lw=2,c='k')

# xx = [-1000,0]
# yy = [23.48,23.48]
# plt.plot(xx,yy,lw=1,ls='-',c='k')

xx = [biotime[-1],biotime[-1]]
yy = [24.2,26.5]
plt.plot(xx,yy,lw=2,c='k')





xx = [biotime[7],biotime[7]]
yy = [28,30.3]
plt.plot(xx,yy,lw=3,c='tab:red')


xx = [biotime[8],biotime[8]]
yy = [28,30.3]
plt.plot(xx,yy,lw=3,c='tab:red')


xx = [biotime[9],biotime[9]]
yy = [28,30.3]
plt.plot(xx,yy,lw=3,c='tab:red')


xx = [biotime[10],biotime[10]]
yy = [28,30.3]
plt.plot(xx,yy,lw=3,c='tab:red')

xx = [biotime[11],biotime[11]]
yy = [28,30.3]
plt.plot(xx,yy,lw=3,c='tab:red')

xx = [biotime[12],biotime[12]]
yy = [28,30.3]
plt.plot(xx,yy,lw=3,c='tab:red')



# Piciforms



xx = [biotime[13],biotime[13]]
yy = [32,34.4]
plt.plot(xx,yy,lw=3,c='tab:blue')


xx = [biotime[14],biotime[14]]
yy = [32,34.4]
plt.plot(xx,yy,lw=3,c='tab:blue')


xx = [biotime[15],biotime[15]]
yy = [32,34.4]
plt.plot(xx,yy,lw=3,c='tab:blue')

xx = [biotime[16],biotime[16]]
yy = [32,34.4]
plt.plot(xx,yy,lw=3,c='tab:blue')

xx = [biotime[17],biotime[17]]
yy = [32,34.4]
plt.plot(xx,yy,lw=3,c='tab:blue')


xx = [biotime[18],biotime[18]]
yy = [32,34.4]
plt.plot(xx,yy,lw=3,c='tab:blue')


rect9 = patches.Rectangle((-1000,27.9),1000,2.6,facecolor='whitesmoke', alpha=1)
rect9s = patches.Rectangle((-1000,27.9),1000,2.6,edgecolor='k', linewidth=1,facecolor='None', alpha=0.5)
ax.add_patch(rect9)
ax.add_patch(rect9s)

rect10 = patches.Rectangle((-1000,31.9),1000,2.6,facecolor='whitesmoke', alpha=1)
rect10s = patches.Rectangle((-1000,31.9),1000,2.6,edgecolor='k', linewidth=1,facecolor='None', alpha=0.5)
ax.add_patch(rect10)
ax.add_patch(rect10s)


# xx = [-1000,0]
# yy = [31.2,31.2]
# plt.plot(xx,yy,lw=1,ls='-',c='k')


# xx = [-1000,0]
# yy = [35.1,35.1]
# plt.plot(xx,yy,lw=1,ls='-',c='k')

# xx = [biotime[6],biotime[6]]
# yy = [18.8,21.2]
# plt.plot(xx,yy,lw=2,c='k')


# xx = [biotime[6],biotime[6]]
# yy = [18.8,21.2]
# plt.plot(xx,yy,lw=2,c='k')


# xx = [biotime[6],biotime[6]]
# yy = [18.8,21.2]
# plt.plot(xx,yy,lw=2,c='k')


# xx = [biotime[-1],biotime[-1]]
# yy = [21.9,24.3]
# plt.plot(xx,yy,lw=2,c='k')


plt.xlim(-1000,0)
plt.ylim(-0.5,35)
plt.xlabel('Time (kyr BP)')
# plt.ylabel('Intraspecific divergence times')
fig.tight_layout()
fig.savefig('bio.pdf')